In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, classification_report, log_loss
from torch.utils.data import Dataset
import PIL
import torchvision.transforms as transforms 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [0]:
###############################################################################
# Dataset Class
class SpectralDataset(Dataset):
    def __init__(self, df):
        
        self.labels = df.label#.str.get_dummies()    #Possible cause no NaNs
        self.paths = df.path
        self.transform = transforms.Compose([transforms.ToTensor()])
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, i):
        #pdb.set_trace()
        path = self.paths.iloc[i]
        X = self.get_spectogram(path)
        y = self.labels.iloc[i]#.values
        return X, y
    
    def get_spectogram(self, path):
        slice_ = PIL.Image.open(path)
        return self.transform(slice_) #Also normalizes the data
    
    def collate_func(self, batch):
        pass

In [3]:

%cd /content/gdrive/My\ Drive/Colab\ Notebooks
!pwd

/content/gdrive/My Drive/Colab Notebooks
/content/gdrive/My Drive/Colab Notebooks


In [4]:
# Load csv here
df = pd.read_csv('spectral_slices.csv')

def label_encoder(df):
    codes = {}
    i = 0
    for label in df.label.drop_duplicates():
        codes['label'] = i
        df.label[df.label == label] = i
        i += 1
    return df

df = label_encoder(df)    

# Get even class representation
sample_size = 400
df = df.groupby('label').apply(lambda x: x.sample(sample_size))

#Split in test and train:
indeces = np.random.permutation(len(df))
split = int(len(indeces) * 0.8)
train_indeces = indeces[:split]
test_indeces = indeces[split:]


train = df.iloc[train_indeces]
test = df.iloc[test_indeces]




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
train.head()

Unnamed: 0 label                      path
label                                                 
1     2923        2923     1  spectral_slices/2924.png
      2973        2973     1  spectral_slices/2974.png
      2760        2760     1  spectral_slices/2761.png
2     2340        2340     2  spectral_slices/2341.png
3     1593        1593     3  spectral_slices/1594.png

In [6]:
classes = 4
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
    
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = nn.Linear(128 * 62 *10, classes)

    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = self.layer3(out)
        #print(out.shape)
        out = self.layer4(out)
        #print(out.shape)

        out = out.reshape(out.size(0), -1)
        #print(out.shape)

        out = self.fc(out)
        return out
    
# Always check your model are you able to make a forward pass and shapes match your expectations?
image = torch.randn(1, 1, 1025, 200)
cnn = SimpleCNN()
output = cnn(image)
print("input shape:")
print(image.shape)
print("output shape:")
print(output.shape)

input shape:
torch.Size([1, 1, 1025, 200])
output shape:
torch.Size([1, 4])


In [7]:
LR= 0.001
BATCH_SIZE = 64


cnn = SimpleCNN()

OPTIMIZER = 'Adam' # one of ['ASGD','Adadelta', 'Adagrad','Adam', 'Adamax','LBFGS', 'RMSprop','Rprop','SGD',SparseAdam']
optimizer = getattr(torch.optim, OPTIMIZER)(cnn.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

cnn.to(DEVICE)

train_ds = SpectralDataset(train)
test_ds = SpectralDataset(test)

# Create dataset loader
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE)

# Lets try to use the criterion with dummy data
yp = torch.randn(BATCH_SIZE, 4)
yt = torch.randint(4, (BATCH_SIZE,))
criterion(yp, yt.long())


tensor(1.6406)

In [8]:
train_ds[0][1]

1

In [0]:
###############################################################################
def evaluate_model(model, test_loader, print_info=False):
    with torch.no_grad():
        model.eval()
        collect_results = []
        collect_target = []
        for batch in test_loader:
            X, y = batch
            X = X.to(DEVICE)
            y = y.to(DEVICE).detach().cpu().numpy()
            pred = cnn(X)
            collect_results.append(F.softmax(pred).detach().cpu().numpy())
            collect_target.append(y) 
    
        preds_proba = np.concatenate(collect_results)
        preds = preds_proba.argmax(axis=1)

        targets = np.concatenate(collect_target)
        
        ll = log_loss(targets, preds_proba)
        acc = accuracy_score(targets, preds)
        if print_info:
            print("test log-loss: {}".format(ll))
            print("overall accuracy:  {}".format(acc))
            #print(classification_report(targets, preds))
        model.train()
        return ll, acc
    
    
            
collect_metrics = []
collect_loss = []
print('Begin training...')
for epoch in range(20):
    lossacc = 0
    for i, batch in enumerate(train_dl):
        #print(f'Batch number {i}')
        optimizer.zero_grad()
        X, y = batch
        X = X.to(DEVICE)
        y = y.to(DEVICE)
        y_pred = cnn(X)
        
        loss = criterion(y_pred, y)
        #print(loss)
        loss.backward()
        optimizer.step()   
        collect_loss.append(float(loss.detach().cpu().numpy()))  
        
    lltest, acctest = evaluate_model(cnn, test_dl)
    lltrain, acctrain = evaluate_model(cnn, train_dl)
    collect_metrics.append([lltest, lltrain, acctest, acctrain])
    print("test: loss: {}  acc: {}".format(lltest, acctest))
    print("train: loss: {}  acc: {}".format(lltrain, acctrain))

Begin training...


In [18]:
print(cnn(test_ds[22][0].unsqueeze(dim = 0).cuda()))
print(test_ds[22][1])

tensor([[ 11.7576,   1.3691, -20.6693,   0.1760]], device='cuda:0',
       grad_fn=<AddmmBackward>)
0


In [7]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 11.5 GB  | Proc size: 2.1 GB
GPU RAM Free: 4069MB | Used: 7372MB | Util  64% | Total 11441MB
